# [Astropy](https://www.astropy.org/)
\
![Astropy](https://docs.astropy.org/en/stable/_static/astropy_banner.svg)
\
`astropy` zawiera mnóstwo funkcji i narzędzi do pracy z danymi astronomicznymi.

W dokumentacji można przeczytać, że pakiet ma funkcje dot.:
* struktura danych i transformacje, m.in: stałe, jednostki, czas i data, współrzędne
* obsługa plików (I/O): tabele ascii, pliki fits, votable
* obliczenia: kosmologia, konwolucje i filtrowanie sygnałów, wizualizacja danych, astrostatystyka

# Spis Treści

1. [Struktura danych](#data_structure)
    * [Stałe i jednostki](#const_and_units)
        - [Ćwiczenie nr 1](#ex1)
    * [Obsługa czasu](#timedate)
    * [Współrzędne](#cooridinates)
2. [Obsługa plików](#files)
    * [Otwieranie plików FITS](#fits)
    * [Wyświetlanie - normalizacja](#norm)
3. [Prosta i szybka fotometria aperturowa](#aperture)

# Importowanie i ustawienie notebook-a (uruchomić jako pierwsze!)

In [ ]:
# import astropy as ap
import numpy as np

# we will need to plot stuff later
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2

<a id=data_structure></a>
# 1. Struktura danych

<a id=const_and_units></a>
## Stałe i jednostki

In [ ]:
import astropy.constants as const
import astropy.units as u

In [ ]:
const.G

In [ ]:
print(const.c)
print("-----------------------")
print(const.c.cgs)
print(type(const.c))
print(const.c.to("pc/yr"))  # konwersja jednostek

In [ ]:
g = (const.G * const.M_earth) / (const.R_earth) ** 2
print(g)

In [ ]:
print(const.g0.value)

In [ ]:
(g*80*u.kg).to(u.N)

In [ ]:
# dir(const)

In [ ]:
# (0.8 * u.arcsec).to(u.parsec)
print((0.5 * u.arcsec).to(u.parsec, equivalencies=u.parallax()))
print((6568 * u.Angstrom).to(u.GHz, equivalencies=u.spectral()))

In [ ]:
grosz = u.def_unit("grosz") # definiowanie własnych jednostek
pln = u.def_unit("pln", 100*grosz)
2.49*pln.to(grosz)

In [ ]:
fortnight = u.def_unit('fortnight', 14 * u.day)
const.c.to("lyr/fortnight")

<a id="ex1"></a>
### Ćwiczenie nr 1

Policz prędkość Ziemi w km/s wokół Słońca (zakładając orbitę kołową). Skorzystaj z poniższego wzoru oraz z modułów `constants` i `units`:

$$v = \sqrt{\frac{G M_{\odot}}{r}}$$

<a id="timedate"></a>
## Obsługa czasu

In [ ]:
from astropy.time import Time

In [ ]:
times = ['1999-01-01T00:00:00.123456789', '2010-01-01T00:00:00']
t = Time(times, format='isot', scale='utc')
t

In [ ]:
t[1]

In [ ]:
t = Time(times)
t

In [ ]:
t.jd

In [ ]:
t.mjd

In [ ]:
nt = Time.now() # current time
print(nt)
nt.sidereal_time('apparent', 'greenwich') 

In [ ]:
nt = Time.now() # current time
t = Time(nt, scale='utc', location=('17d', '51.1d'))
print(t)
t.sidereal_time('apparent')

<a id="coordinates"></a>
## Współrzędne

In [ ]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun, get_moon

`astropy.coordinates.SkyCoord.from_name` używa Simbad-a do wyszkiwania obiektów po nazwie

Wyszukajmy współrzędne dla NGC6823:

In [ ]:
obj_name = "NGC6823"
obj = SkyCoord.from_name(obj_name)

In [ ]:
obj

Użyjmy `astropy.coordinates.EarthLocation` żeby stworzyć obiekt `location` z danymi dot. miejsca obserwacji.
Weźmy obserwatorium w Białkowie:


In [ ]:
bialkow = EarthLocation(lat=51.476111*u.deg, lon=16.656667*u.deg, height=130*u.m)
utcoffset = 1*u.hour  # Obecnie mamy czas zimowy
time = Time.now()
time

Skonwertujmy współrzędne do `AltAz`(ymutalnych) w miejscu obserwacji

In [ ]:
obj_altaz = obj.transform_to(AltAz(obstime=time,location=bialkow))
print(obj_altaz.alt)

In [ ]:
y, m, d, _, _, _ = time.ymdhms
midnight = Time(str(y)+'-'+str(m)+'-'+str(d)+' 00:00:00') - utcoffset
delta_midnight = np.linspace(-12, 12, 1000)*u.hour
delta_midnight
night = AltAz(obstime=midnight+delta_midnight, location=bialkow)
obj_altaz = obj.transform_to(night)

In [ ]:
plt.plot(delta_midnight, obj_altaz.alt)
plt.xlim(-12, 12)
plt.ylim(-20, 65)
plt.xlabel('Hours from midnight [h]')
plt.ylabel('Altitude [deg]')
plt.show()

In [ ]:
sun_altaz = get_sun(midnight+delta_midnight).transform_to(night)
moon_altaz = get_moon(midnight+delta_midnight).transform_to(night)

In [ ]:
plt.plot(delta_midnight, obj_altaz.alt, lw=3, label=obj_name)
plt.plot(delta_midnight, sun_altaz.alt, color="orange", ls="--", lw=1, label="Sun")
plt.plot(delta_midnight, moon_altaz.alt, color="gray", ls="--", lw=1, label="Moon")
plt.fill_between(delta_midnight, -90*u.deg, 90*u.deg, sun_altaz.alt < -0*u.deg, color='0.75')
plt.fill_between(delta_midnight, -90*u.deg, 90*u.deg, sun_altaz.alt < -6*u.deg, color='0.5')
plt.fill_between(delta_midnight, -90*u.deg, 90*u.deg, sun_altaz.alt < -12*u.deg, color='0.25')
plt.fill_between(delta_midnight, -90*u.deg, 90*u.deg, sun_altaz.alt < -18*u.deg, color='k')
plt.xlim(-12, 12)
plt.ylim(-90, 90)
plt.xlabel('Hours from midnight [h]')
plt.ylabel('Altitude [deg]')
plt.legend(loc='lower left')
plt.show()

# 2. Obsługa plików

<a id="fits"></a>
## Otwieranie plików FITS

In [ ]:
import os
from astropy.io import fits
import matplotlib.pyplot as plt

In [ ]:
dirpath = os.path.join("..","data")
inputfile = "NGC6823_UDF_I.fits"
data, header = fits.getdata(os.path.join(dirpath, inputfile), header=True)

In [ ]:
fitsfile = fits.open(os.path.join(dirpath, inputfile))

In [ ]:
print(fitsfile.info())

In [ ]:
print(fitsfile[0].data)

In [ ]:
print(fitsfile[0].header)

In [ ]:
# with fits.open(os.path.join(dirpath, inputfile), mode='update') as filehandle:
#     filehandle[0].header.pop('MYHDRKW')
#     filehandle[0].header['MYHDRKW']='My Header Keyword'

In [ ]:
plt.imshow(fitsfile[0].data, cmap='gray')
plt.colorbar()

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
plt.imshow(fitsfile[0].data, cmap='gray', norm=LogNorm())

In [ ]:
from astropy.visualization import MinMaxInterval, SqrtStretch, ImageNormalize, ZScaleInterval

norm = ImageNormalize(fitsfile[0].data, interval=ZScaleInterval(), stretch=SqrtStretch())
plt.imshow(fitsfile[0].data, cmap='gray', norm=norm)

<a id="norm"></a>
## Wyświetlanie - normalizacja

In [ ]:
image_list = [os.path.join(dirpath,"M13_blue_000"+str(n)+".fits") for n in range(1, 5+1)]
image_concat = [fits.getdata(image) for image in image_list]

In [ ]:
image_concat

In [ ]:
final_image = np.zeros(shape=image_concat[0].shape)

In [ ]:
for image in image_concat:
    final_image += image

In [ ]:
final_image

In [ ]:
from astropy.visualization.stretch import SinhStretch, LinearStretch

norm = ImageNormalize(image_concat[0], interval=ZScaleInterval(), stretch=SinhStretch())

plt.imshow(image_concat[0], cmap='gray', norm=norm)

In [ ]:
norm = ImageNormalize(final_image, interval=ZScaleInterval(), stretch=SinhStretch())

plt.imshow(final_image, cmap='gray', norm=norm)

<a id="aperture"></a>
# Prosta i szybka fotometria aperturowa

In [ ]:
inputfile = "HST.fits"
hst = fits.open(os.path.join(dirpath, inputfile))
hst_image = hst[0].data[0]
hst_image -= np.median(hst_image)

In [ ]:
norm = ImageNormalize(hst_image, interval=ZScaleInterval(), stretch=SqrtStretch())
plt.imshow(hst_image, cmap='gray', norm=norm)

In [ ]:
# conda install -c conda-forge photutils
# pip install photutils

from photutils.detection import DAOStarFinder
from astropy.stats import mad_std
bkg_sigma = mad_std(hst_image)  
daofind = DAOStarFinder(fwhm=1., threshold=3. * bkg_sigma)  
sources = daofind(hst_image)  
for col in sources.colnames:  
    sources[col].info.format = '%.8g'  # for consistent table output
print(sources)  

In [ ]:
from photutils.aperture import aperture_photometry, CircularAperture
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))  
apertures = CircularAperture(positions, r=3.)  
phot_table = aperture_photometry(image, apertures)  
for col in phot_table.colnames:  
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(hst_image, cmap='gray', origin='lower', norm=norm)
apertures.plot(color='blue', lw=1.5, alpha=0.5);

# [Astroquery](https://astroquery.readthedocs.io/en/latest/)

### Simbad

In [ ]:
from astroquery.simbad import Simbad

result_table = Simbad.query_object("NGC6823")
result_table.pprint()

In [ ]:
result_table

In [ ]:
from astropy import coordinates
import astropy.units as u

c = coordinates.SkyCoord("05h35m17.3s -05d23m28s", frame='icrs')

r = 5 * u.arcminute

result_table = Simbad.query_region(c, radius=r)

result_table.pprint(show_unit=True, max_width=80, max_lines=5)

In [ ]:
# result_table

In [ ]:
result_table = Simbad.query_object("m [1-9]", wildcard=True)

print(result_table)

### Vizier

In [ ]:
from astroquery.vizier import Vizier
from astropy.coordinates import Angle
from astroquery.vizier import Conf
Conf.row_limit = -1
# Vizier.ROW_LIMIT = -1

result = Vizier.query_region("NGC6823", radius=Angle(0.1, "deg"), catalog='I/355/gaiadr3')
# result = Vizier.query_region("NGC6823", radius="0d6m0s", catalog='I/355/gaiadr3')
# result = Vizier.query_region("NGC6823", radius=0.1*u.deg, catalog='I/355/gaiadr3')
result

In [ ]:
result = Vizier(catalog="III/135A", columns=["HD","_RA.icrs","_DEC.icrs","Ptg"]).query_constraints(Ptg="5.0..6.0")
result[0]